In [1]:
import os
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data'
os.chdir(path)

today = datetime.now().strftime("%Y-%m-%d")
yesterday = datetime.strftime(datetime.now() - timedelta(1), "%Y-%m-%d")

csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data\csv.csv'
f = open(csv,'w+')

xl_file = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\meta_files\colocation_metadata.xlsx'

In [3]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3], axis=1)

In [4]:
try:
    print('Creating a new directory for today...')
    os.mkdir(path + r'\five_minute\five_minute_' + today)
    new_folder = (r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data\five_minute\five_minute_'
    + today)
except Exception as e:
    print('Something went wrong while creating a new directory for today: ' + e.args[0])
else:
    print('New directory successfully created!')

Creating a new directory for today...
New directory successfully created!


In [10]:
def fivemin_nullcheck(name):
    today = datetime.now().strftime("%Y-%m-%d")
    yesterday = datetime.strftime(datetime.now() - timedelta(1), "%Y-%m-%d")
    try:
        print('Creating a new text file for ' + str(name) + '...')
        txt = new_folder + r'\null_report' + str(name) + str(today)
        g = open(txt,'a')
    except Exception as e:
        print('Something went wrong while creating a new text file for ' + str(name) + ': ' + e.args[0])
    try:
        print('Pulling five minute data from ' + str(name) + '...')
        urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=five_minute&sids='
                                   + str(name) + '&sdate=' + str(yesterday) + '&edate=' + str(today)
                                   + '&elems=tmean,rh,vp,sr,ws,wind_vec,pp,st5,st15,gust,gusttm,gustdir', filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
        df.columns = ['station','date','tmean','rh','vp','sr','ws','wind_vec','pp','st5','st15','gust','gusttm','gustdir']
        df.drop(df.tail(1).index,inplace=True)
    except ValueError:
        print(str(name) + ' does not report 5-minute data. The function will move to the next station...')
        return
    except Exception as e:
            print('Soemthing went wrong while pullitng the 5-minute data from' + str(name) + ': ' + e.args[0])
    try:
        print('Repopulating the 5-minute data from ' + str(name) + '...')
        df['date'] = pd.to_datetime(df.date)
        r = pd.date_range(start=yesterday,end=today,freq='5min')
        df = df.set_index('date').reindex(r,copy=False).rename_axis('date')
    except Exception as e:
        print('Something went wrong while repopulating the 5-minute data from ' + str(name) + ': ' + e.args[0])
    try:
        print('Checking the 5-minute data from ' + str(name) + ' for missing values...')
        d = 0
        for index, row in df.iterrows():
            if(pd.isnull(df['tmean'][d])):
                tmean_string = (str(df.station[d]) + ' reported missing 5-minute temperature data at ' +  str(df.index[d]) + '\n')
                g.write(tmean_string)
            elif(pd.isnull(df['rh'][d])):
                rh_string = (str(df.station[d]) + ' reported missing 5-minute relative humidity data at ' + str(df.index[d]) + '\n')
                g.write(rh_string)
            elif(pd.isnull(df['vp'][d])):
                vp_string = (str(df.station[d]) + ' reported missing 5-minute vapor pressure data at ' + str(df.index[d]) + '\n')
                g.write(vp_string)
            elif(pd.isnull(df['sr'][d])):
                sr_string = (str(df.station[d]) + ' reported missing 5-minute solar radiation data at ' + str(df.index[d]) + '\n')
                g.write(sr_string)
            elif(pd.isnull(df['ws'][d])):
                ws_string = (str(df.station[d]) + ' reported missing 5-minute wind speed data at ' + str(df.index[d]) + '\n')
                g.write(ws_string)
            elif(pd.isnull(df['wind_vec'][d])):
                windvec_string = (str(df.station[d]) + ' reported missing 5-minute wind direction data at ' + str(df.index[d]) + '\n')
                g.write(windvec_string)
            elif(pd.isnull(df['pp'][d])):
                pp_string = (str(df.station[d]) + ' reported missing 5-minute precipitation data at ' + str(df.index[d]) + '\n')
                g.write(pp_string)
            elif(pd.isnull(df['st5'][d])):
                st5_string = (str(df.station[d]) + ' reported missing 5-minute st5 data at ' + str(df.index[d]) + '\n')
                g.write(st5_string)
            elif(pd.isnull(df['st15'][d])):
                st15_string = (str(df.station[d]) + ' reported missing 5-minute st15 data at ' + str(df.index[d]) + '\n')
                g.write(st15_string)
            elif(pd.isnull(df['gust'][d])):
                gust_string = (str(df.station[d]) + ' reported missing 5-minute gust data at ' + str(df.index[d]) + '\n')
                g.write(gust_string)
            elif(pd.isnull(df['gusttm'][d])):
                gusttm_string = (str(df.station[d]) + ' reported missing 5-minute gust time data at ' + str(df.index[d]) + '\n')
                g.write(gusttm_string)
            elif(pd.isnull(df['gustdir'][d])):
                gustdir_string = (str(df.station[d]) + ' reported missing 5-minute gust time data at ' + str(df.index[d]) + '\n')
                g.write(gustdir_string)
            d=d+1
    except Exception as e:
        print('Something went wrong while checking the data from ' + str(name) + ': ' + e.args[0])

In [11]:
n=0
for index,rows in meta.iterrows():
    fivemin_nullcheck(str(meta.ID[n]))
    n=n+1

Creating a new text file for AKR02...
Pulling five minute data from AKR02...
AKR02 does not report 5-minute data...
Creating a new text file for ALT01...
Pulling five minute data from ALT01...
ALT01 does not report 5-minute data...
Creating a new text file for AVN01...
Pulling five minute data from AVN01...
Repopulating the 5-minute data from AVN01...
Checking the 5-minute data from AVN01 for missing values...
Creating a new text file for BNV01...
Pulling five minute data from BNV01...
Repopulating the 5-minute data from BNV01...
Checking the 5-minute data from BNV01 for missing values...
Creating a new text file for BRG01...
Pulling five minute data from BRG01...
BRG01 does not report 5-minute data...
Creating a new text file for BRK01...
Pulling five minute data from BRK01...
BRK01 does not report 5-minute data...
Creating a new text file for BRL02...
Pulling five minute data from BRL02...
Repopulating the 5-minute data from BRL02...
Checking the 5-minute data from BRL02 for missing 

Repopulating the 5-minute data from MTR01...
Checking the 5-minute data from MTR01 for missing values...
Creating a new text file for ORM01...
Pulling five minute data from ORM01...
ORM01 does not report 5-minute data...
Creating a new text file for OTH01...
Pulling five minute data from OTH01...
Repopulating the 5-minute data from OTH01...
Checking the 5-minute data from OTH01 for missing values...
Creating a new text file for PAI01...
Pulling five minute data from PAI01...
PAI01 does not report 5-minute data...
Creating a new text file for PKH01...
Pulling five minute data from PKH01...
Repopulating the 5-minute data from PKH01...
Checking the 5-minute data from PKH01 for missing values...
Creating a new text file for PKR01...
Pulling five minute data from PKR01...
PKR01 does not report 5-minute data...
Creating a new text file for PNR01...
Pulling five minute data from PNR01...
PNR01 does not report 5-minute data...
Creating a new text file for RFD01...
Pulling five minute data from